In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import seaborn as sns
driving = pd.read_csv("/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/garminlongdrive.csv")
walking=pd.read_csv("/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/sergiolongwalk.csv")
driving2=pd.read_csv("/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/sergioparkinglot.csv")

import statistics as stats
def addvariance(tracking_df):
    vartwo=[]
    varfive=[]
    varten=[]
    for i,j in enumerate(tracking_df.mph):
        var=0
        if i>=2:
            previoustwo = [j,tracking_df.mph[i-1]]
            var=stats.variance(previoustwo)
        vartwo.append(var)
        if i>=5:
            previousfive = [j,tracking_df.mph[i-1],tracking_df.mph[i-2],tracking_df.mph[i-3],tracking_df.mph[i-4]]
            var=stats.variance(previousfive)
        varfive.append(var)
        if i>=10:
            previousten = [j,tracking_df.mph[i-1],tracking_df.mph[i-2],tracking_df.mph[i-3],tracking_df.mph[i-4],tracking_df.mph[i-5],tracking_df.mph[i-6],tracking_df.mph[i-7],tracking_df.mph[i-9]]
            var=stats.variance(previousten)
        
        varten.append(var)
    tracking_df["variance2"]=vartwo
    tracking_df["variance5"]=varfive
    tracking_df["variance10"]=varten
    
addvariance(walking)
addvariance(driving)
addvariance(driving2)
driving["Class"]=1
driving2["Class"]=1
walking["Class"]=0
tracking_df=walking.append(driving[0:500]).append(driving2)
tracking_df=tracking_df.reset_index(drop=True)
tracking_df=tracking_df.drop(["latitude","longitude","time","distanceinmeters","HHMMSS","meters","seconds","heartrate"],axis=1)
import matplotlib.pyplot as plt  
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import normalize
def modelrunner(df):
    randomized = (df.sample(frac=1).reset_index(drop=True)).to_numpy()

    randomized = normalize(randomized, norm='max', axis=0)
    X=randomized[:,:-1]
    y=randomized[:,-1:][:,0] 
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=1000/(len(y)), random_state=42)

    svm =  SVC(max_iter=5000)

    search_space = [{'kernel': ['linear'],
                    'C': np.logspace(-7,3,11)},
                    {'kernel': ['poly'],
                    'degree': [2,3],
                    'C':np.logspace(-7,3,11)},
                    {'kernel': ['rbf'],
                    'gamma': [0.001,0.005,0.01,0.05,0.1,0.5,1,2],
                    'C': np.logspace(-7,3,11)}
                    ]

    # Create grid search 
    clf = GridSearchCV(svm, search_space, scoring=['accuracy'],cv =5 ,refit=False, verbose=0)

    # Fit grid search
    best_model = clf.fit(X_train, Y_train) 
    best_model
    param= best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ]
    print("best parameters are ",param)

    svmtest= SVC()
    svmtest.set_params(**param)
    svmtest.fit(X_train,Y_train)
    Y_pred=svmtest.predict(X_test)
    print("accuracy score is ", metrics.accuracy_score(Y_test,Y_pred))

    return svmtest
svm = modelrunner(tracking_df)
def classifier(tracking_df):
    df=tracking_df.drop(["latitude","longitude","time","distanceinmeters","HHMMSS","meters","seconds","heartrate"],axis=1)
    test=normalize(df.to_numpy(), norm='max', axis=0)
    y=svm.predict(test)
    print(len(y))
    tracking_df["Transport"]=y
    tracking_df=tracking_df.drop(["distanceinmeters","heartrate","HHMMSS","meters","seconds","variance2","variance5","variance10"],axis=1)
    return tracking_df


best parameters are  {'C': 1000.0, 'gamma': 2, 'kernel': 'rbf'}
accuracy score is  1.0


# Upload tracking tcx files here to classify transportation type and add speed

In [6]:
import ipywidgets as widgets
import pandas as pd
import codecs
import xml.etree.ElementTree as ET 
import os

In [ ]:
upload=widgets.FileUpload(
    accept='.tcx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
upload

FileUpload(value={}, accept='.tcx', description='Upload')

In [ ]:
name=list(upload.value)[0]
byte=upload.value[name]["content"]
txt=codecs.decode(byte, encoding="utf-8")
xroot = ET.fromstring(txt)

df_cols = ["time","latitude","longitude","distanceinmeters","heartrate"]
rows = []
ctr=0
for lap in xroot[0][0].findall("{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Lap"):
    #should be lap[8] but dunno why this works
    for node in lap[6].findall("{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Trackpoint"):
        if node[1].tag == "{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position":
            time = node[0].text
            lat = node[1][0].text
            long = node[1][1].text
            dist= node[3].text
            bpm = node[4][0].text

            rows.append({"time": time, "latitude": lat, 
                            "longitude": long, "distanceinmeters": dist,
                            "heartrate":bpm})

    tracking_df = pd.DataFrame(rows, columns = df_cols)
    
def time_splitter (row):
   datetime=row['time'].split("T")
   return datetime[1]
  
tracking_df["HHMMSS"]=tracking_df.apply (lambda row: time_splitter(row), axis=1)

tracking_df.distanceinmeters=tracking_df.distanceinmeters.astype(str).astype(float)

meterscol=[]
for i,j in enumerate(tracking_df.distanceinmeters):
    meters=0
    if i>0:
        lastmet=tracking_df.distanceinmeters[i-1]
        met =j
        meters=met-lastmet
    meterscol.append(meters)


tracking_df["meters"]=meterscol

secondscol=[]
for i,j in enumerate(tracking_df.HHMMSS):
    seconds=0
    if i>0:
        lastmins=int(tracking_df.HHMMSS[i-1][3:5])
        lastsecs=int(tracking_df.HHMMSS[i-1][6:8])
        mins=int(j[3:5])
        secs=int(j[6:8])
        if lastmins==mins:
            seconds=secs-lastsecs
        else:
            seconds = (60-lastsecs)+secs
    secondscol.append(seconds)
    
tracking_df["seconds"]=secondscol

def calculate_speed(row):
    if (row.seconds>0):
        meterspersecond=row.meters/row.seconds
        milesperhour=meterspersecond*2.237
        return milesperhour
    else: return 0
    
tracking_df["mph"]=tracking_df.apply (lambda row: calculate_speed(row), axis=1)
addvariance(tracking_df)
tracking_df = tracking_df[tracking_df.mph < 85]
tracking_df.head()

testingclassified=classifier(tracking_df)
testingclassified.head()
testingclassified[['date','hour']] = testingclassified.time.str.split("T",expand=True)
testingclassified.head()
testingclassified['hour'] = testingclassified['hour'].str.replace('.000Z','')
testingclassified.head()
testingclassified['timeOfDay'] = pd.cut(pd.to_datetime(testingclassified.hour).dt.hour,
       bins=[0, 6, 12, 18, 24],
       labels=['night', 'morning', 'afternoon', 'evening'],
       right=False,
       include_lowest=True)
#filename= "testing1" 
filename = name.split(".")[0] #filename you can change 
print(filename)
import os

os.makedirs('/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Machine learning/Notebooks/classified data'+'/'+ name)

newpath = '/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Machine learning/Notebooks/classified data'+'/'+ name + '/'
testingclassified.head()
testingclassified.to_csv(r"/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Machine learning/Notebooks/classified data/"+name+".csv", index = False)
for (date), group in testingclassified.groupby(['date']):
     group.to_csv(newpath + filename + f'{date}.csv', index=False)

NameError: ignored

In [16]:
#only use this if not upload locally
import os
import pandas as pd 
import xml.etree.ElementTree as et 

Watchname = 'UCSD13:MCAS19'

xtree = et.parse('/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/DataFromWatch/' + Watchname + '.tcx')
xroot = xtree.getroot() 

df_cols = ["time","latitude","longitude","distanceinmeters","heartrate"]
rows = []


ctr=0
for lap in xroot[0][0].findall("{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Lap"):
    for node in lap[6].findall("{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Trackpoint"):
        if node[1].tag == "{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position":
            time = node[0].text
            lat = node[1][0].text
            long = node[1][1].text
            dist= node[3].text
            bpm = node[4][0].text

            rows.append({"time": time, "latitude": lat, 
                            "longitude": long, "distanceinmeters": dist,
                            "heartrate":bpm})

    tracking_df = pd.DataFrame(rows, columns = df_cols)
def time_splitter (row):
   datetime=row['time'].split("T")
   return datetime[1]
tracking_df["HHMMSS"]=tracking_df.apply (lambda row: time_splitter(row), axis=1)
tracking_df.distanceinmeters=tracking_df.distanceinmeters.astype(str).astype(float)
meterscol=[]
for i,j in enumerate(tracking_df.distanceinmeters):
    meters=0
    if i>0:
        lastmet=tracking_df.distanceinmeters[i-1]
        met =j
        meters=met-lastmet
    meterscol.append(meters)
tracking_df["meters"]=meterscol
secondscol=[]
for i,j in enumerate(tracking_df.HHMMSS):
    seconds=0
    if i>0:
        lastmins=int(tracking_df.HHMMSS[i-1][3:5])
        lastsecs=int(tracking_df.HHMMSS[i-1][6:8])
        mins=int(j[3:5])
        secs=int(j[6:8])
        if lastmins==mins:
            seconds=secs-lastsecs
        else:
            seconds = (60-lastsecs)+secs
    secondscol.append(seconds)
tracking_df["seconds"]=secondscol
def calculate_speed(row):
    if (row.seconds>0):
        meterspersecond=row.meters/row.seconds
        milesperhour=meterspersecond*2.237
        return milesperhour
    else: return 0

tracking_df["mph"]=tracking_df.apply (lambda row: calculate_speed(row), axis=1)
tracking_df.to_csv(r'/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/convertedData/'+ Watchname + '.csv', index = False)
#only use this if not upload locally
filepath = '/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/convertedData/'+ Watchname + '.csv'
tracking_df = pd.read_csv (filepath)
name = os.path.basename(filepath)
addvariance(tracking_df)

testingclassified=classifier(tracking_df)
testingclassified.head()
testingclassified[['date','hour']] = testingclassified.time.str.split("T",expand=True)
testingclassified.head()
testingclassified['hour'] = testingclassified['hour'].str.replace('.000Z','')
testingclassified.head()
testingclassified['timeOfDay'] = pd.cut(pd.to_datetime(testingclassified.hour).dt.hour,
       bins=[0, 6, 12, 18, 24],
       labels=['night', 'morning', 'afternoon', 'evening'],
       right=False,
       include_lowest=True)
#filename= "testing1" 



testingclassified['destination'] = ''
i=0
for i in range(3,len(testingclassified)):
    if abs(testingclassified['longitude'][i] - testingclassified['longitude'][i - 3]) <= 0.00001:
      if testingclassified['Transport'][i] == 0:
        testingclassified['destination'][i] = "1"
        testingclassified['destination'][i-1] = "1"
        testingclassified['destination'][i-2] = "1"
        testingclassified['destination'][i-3] = "1"
        i= i+3
      else:
        testingclassified['destination'][i] = "0"
        testingclassified['destination'][i-1] = "0"
        testingclassified['destination'][i-2] = "0"
        testingclassified['destination'][i-3] = "0"
        i = i+3
    else:
        testingclassified['destination'][i] = "0"
        testingclassified['destination'][i-1] = "0"
        testingclassified['destination'][i-2] = "0"
        testingclassified['destination'][i-3] = "0"
        i = i+3

filename = name.split(".")[0] #filename you can change 
print(filename)
import os

os.makedirs('/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/classified data'+'/'+ filename)

newpath = '/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/classified data'+'/'+ filename + '/'
testingclassified.head()
testingclassified.to_csv(r"/content/drive/Shareddrives/APG_Smart Installation Mobility- Team Access/Sub-Theme Tech & ArcGIS /Machine learning/Notebooks/classified data/"+filename+".csv", index = False)
for (date), group in testingclassified.groupby(['date']):
     group.to_csv(newpath + filename + f'{date}.csv', index=False)

3350


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying 

UCSD13:MCAS19
